In [1]:
import os
import pandas as pd
from langchain_community.document_loaders import CSVLoader
from langchain_openai import AzureOpenAI, AzureOpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.schema import Document

# Récupérer les clés API d'Azure
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY_4')
azure_openai_api_endpoint = os.getenv('AZURE_OPENAI_API_ENDPOINT_4')
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

# Modèle d'embeddings
embedding_model = AzureOpenAIEmbeddings(
    openai_api_key=azure_openai_api_key,
    azure_deployment='text-embedding-3-large',
    azure_endpoint=azure_openai_api_endpoint,
    openai_api_version="2023-05-15",
    chunk_size=500
)

In [2]:
model = AzureChatOpenAI(api_key=azure_openai_api_key,
                        api_version="2023-12-01-preview",
                        azure_endpoint=azure_openai_api_endpoint,
                        model=deployment_name,
                        temperature=0
                        )


NameError: name 'AzureChatOpenAI' is not defined

In [17]:
model.invoke("Couleur du cheval de Napoléon ?")

AIMessage(content="Le cheval le plus célèbre de Napoléon était blanc et s'appelait Marengo.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 16, 'total_tokens': 34, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_67802d9a6d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-c1aceffc-7229-4d5d-ad24-571f1e677114-0', usage_metadata={'input_tokens': 16, 'output_tokens': 18, 'total_tokens': 34})

In [3]:
def load_books(file_path):
    # Charger les livres depuis un fichier CSV
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
        # Créer une liste de documents à partir des colonnes du CSV
        documents = [
            Document(page_content=row['summary'], metadata={"title": row['title'], "author": row['author']})
            for _, row in df.iterrows()
        ]
    else:
        raise ValueError("Unsupported file format. Please use a CSV file.")
    
    return documents

In [4]:
books_file_path = "cleaned_books.csv"
documents = load_books(books_file_path)

# Créer un index vectoriel avec InMemoryVectorStore
index = InMemoryVectorStore.from_documents(
    documents,
    embedding=embedding_model  # Passer l'instance de l'embedding
)

In [18]:
def create_agent():
    qa_prompt_template = PromptTemplate(
        input_variables=["question", "documents"],
        template="""Answer the question '{question}' based on the following documents: {documents}"""
    )
    
    # Créer la chaîne de question-réponse
    qa_chain = RetrievalQA.from_chain_type(
        llm=model,  # Utiliser le modèle de langage ici
        chain_type="stuff",
        retriever=index.as_retriever()
    )
    return qa_chain

def ask_question(question):
    agent = create_agent()
    response = agent.invoke(question)
    return response

In [19]:
question = "Balzac a écrit quoi ?"
answer = ask_question(question)
print(f"Réponse : {answer}")

Réponse : {'query': 'Balzac a écrit quoi ?', 'result': 'Honoré de Balzac a écrit une vaste série de romans intitulée "La Comédie Humaine," qui explore divers aspects de la société française du 19ème siècle. Parmi ses œuvres notables, on trouve "Le Père Goriot," "Eugénie Grandet," "Illusions perdues," et "Splendeurs et misères des courtisanes." Ses écrits abordent des thèmes tels que l\'ambition, l\'amour, la société, et l\'économie.'}


In [7]:
llm_model.invoke("Couleur du soleil ?")

BadRequestError: Error code: 400 - {'error': {'code': 'OperationNotSupported', 'message': 'The completion operation does not work with the specified model, gpt-4o. Please choose different model and try again. You can learn more about which models can be used with each operation here: https://go.microsoft.com/fwlink/?linkid=2197993.'}}